#### 2. PyTorch II

#### 2-1. download data and unzip it

In [1]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1LE1fsiN4kw8F_eD88l6xN2XyOXLiANZI
!unzip -q data.zip

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
Downloading...
From (original): https://drive.google.com/uc?id=1LE1fsiN4kw8F_eD88l6xN2XyOXLiANZI
From (redirected): https://drive.google.com/uc?id=1LE1fsiN4kw8F_eD88l6xN2XyOXLiANZI&confirm=t&uuid=54ea3b5f-c019-4fb9-86f9-a6ddf17346c2
To: /home/loveofmylife/Tutorial_PyTorch/data.zip
100%|██████████████████████████████████████| 1.24G/1.24G [01:52<00:00, 11.0MB/s]


#### 2-2. Prepare dataset I

In [1]:
import pandas as pd
import torch
import torch.nn as nn

In [2]:
dataset = pd.read_csv(r'chap02/car_evaluation.csv')
dataset.head()

,price,maint,doors,persons,lug_capacity,safety,output
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [3]:
categorical_columns = ['price', 'maint', 'doors', 'persons', 'lug_capacity', 'safety']

tensor_list = []
for category in categorical_columns:
    dataset[category] = dataset[category].astype('category')
    tensor_list.append(torch.Tensor(dataset[category].cat.codes.values).to(dtype=torch.int64))

categorical_data = torch.stack(tensor_list, dim=1)
categorical_data[:10]

/tmp/ipykernel_2312061/753236517.py:6: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /opt/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:199.)
  tensor_list.append(torch.Tensor(dataset[category].cat.codes.values).to(dtype=torch.int64))


tensor([[3, 3, 0, 0, 2, 1],
        [3, 3, 0, 0, 2, 2],
        [3, 3, 0, 0, 2, 0],
        [3, 3, 0, 0, 1, 1],
        [3, 3, 0, 0, 1, 2],
        [3, 3, 0, 0, 1, 0],
        [3, 3, 0, 0, 0, 1],
        [3, 3, 0, 0, 0, 2],
        [3, 3, 0, 0, 0, 0],
        [3, 3, 0, 1, 2, 1]])

In [4]:
# 1.torch.arange(0,5,1)를 이용해 Tensor를 생성하고 해당 텐서의 데이터 형태를 torch.float32 로 바꾸기
torch.arange(0,5,1).to(dtype=torch.float32)

tensor([0., 1., 2., 3., 4.])

In [5]:
# 2. stack 메서드를 이용하고 torch.tensor([1.,3.,6.,10.]) 와 torch.tensor([2.,7.,9.,13.])를 쌓아
# tensor([[ 1.,  2.], [ 3.,  7.], [ 6.,  9.], [10., 13.]]) 만들기
torch.stack([torch.tensor([1.,3.,6.,10.]), torch.tensor([2.,7.,9.,13.])], dim=1)

tensor([[ 1.,  2.],
        [ 3.,  7.],
        [ 6.,  9.],
        [10., 13.]])

In [6]:
# 3.cat 메서드를 이용하여 torch.ones(5,7,8), torch.zeros(5,4,8) 를 합쳐 모양이 (5,11,8) 인 텐서를 생성하기
torch.cat([torch.ones(5,7,8), torch.zeros(5,4,8)], dim=1).shape

torch.Size([5, 11, 8])

#### 2-3. Prepare dataset II

In [7]:
outputs = pd.get_dummies(dataset.output)
outputs = outputs.values
# outputs = torch.tensor(outputs).flatten()
outputs = torch.tensor(outputs)

print(categorical_data.shape)
print(outputs.shape)

torch.Size([1728, 6])
torch.Size([1728, 4])


In [8]:
categorical_column_sizes = [len(dataset[column].cat.categories) for column in categorical_columns]
categorical_embedding_sizes = [(col_size, min(50, (col_size+1)//2)) for col_size in categorical_column_sizes]
print(categorical_embedding_sizes)

[(4, 2), (4, 2), (4, 2), (3, 2), (3, 2), (3, 2)]


In [9]:
total_records = 1728
test_records = int(total_records * .2)

categorical_train_data = categorical_data[:total_records-test_records]
categorical_test_data = categorical_data[total_records-test_records:total_records]
train_outputs = outputs[:total_records-test_records]
test_outputs = outputs[total_records-test_records:total_records]

In [10]:
print(len(categorical_train_data))
print(len(train_outputs))
print(len(categorical_test_data))
print(len(test_outputs))

1383
1383
345
345


In [11]:
# 1.for loop를 list comprehension을 이용해 한 줄로 만들어라.
fruits = ["apple", "banana", "cherry", "kiwi", "mango"]
newlist = []

for x in fruits:
  if "a" in x:
    newlist.append(x)

print(newlist)

# answer
fruits = ["apple", "banana", "cherry", "kiwi", "mango"]
[x for x in fruits if 'a' in x]

['apple', 'banana', 'mango']


['apple', 'banana', 'mango']

In [12]:
# 2.아래와 같이 정의된 my_column_sizes를 이용해 워드 임베딩을 하려고 한다.
# 이를 이용하여 임베딩 크기(차원)을 구하는 코드를 만들어라.
# 임베딩 크기는 (column_size+1)/2 와 30 중 작은 수로 만들려고 한다.
my_column_sizes = [1, 2, 34, 15, 59, 34, 13]

# answer
my_column_sizes = [1, 2, 34, 15, 85, 34, 13]
[(size, min(30, int((size+1)//2)))for size in my_column_sizes]

[(1, 1), (2, 1), (34, 17), (15, 8), (85, 30), (34, 17), (13, 7)]

#### 2-4. Define model

In [59]:
# 1.nn.Linear를 이용해 길이 5인 벡터를 길이 10인 벡터로 늘릴 수 있게 만들어 보자.
# 2.nn.ReLU와 torch.abs 함수와의 차이가 있을까?
# 3.nn.Dropout 을 생성하고 특성을 확인해보자. 매번 값이 바뀌는가?

In [58]:
import torch.nn as nn
net = nn.Linear(5, 10)
x = torch.ones(5)
# net(x)

net = nn.ReLU()
net(-x), torch.abs(-x)

(tensor([0., 0., 0., 0., 0.]), tensor([1., 1., 1., 1., 1.]))

In [57]:
class Model(nn.Module):
    def __init__(self, embedding_size, output_size, layers, p=0.4):
        super().__init__()
        self.all_embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in embedding_size])
        self.embedding_dropout = nn.Dropout(p)

        all_layers = []
        num_categorical_cols = sum((nf for ni, nf in embedding_size))
        input_size = num_categorical_cols

        for i in layers:
            all_layers.append(nn.Linear(input_size, i))
            all_layers.append(nn.ReLU(inplace=True))
            all_layers.append(nn.BatchNorm1d(i))
            all_layers.append(nn.Dropout(p))
            input_size = i

        all_layers.append(nn.Linear(layers[-1], output_size))
        self.layers = nn.Sequential(*all_layers)

    def forward(self, x_categorical):
        embeddings = []
        for i,e in enumerate(self.all_embeddings):
            embeddings.append(e(x_categorical[:,i]))
        x = torch.cat(embeddings, 1)
        x = self.embedding_dropout(x)
        x = self.layers(x)
        return x

In [56]:
model = Model(categorical_embedding_sizes, 4, [200,100,50], p=0.4)
print(model)

Model(
  (all_embeddings): ModuleList(
    (0): Embedding(4, 2)
    (1): Embedding(4, 2)
    (2): Embedding(4, 2)
    (3): Embedding(3, 2)
    (4): Embedding(3, 2)
    (5): Embedding(3, 2)
  )
  (embedding_dropout): Dropout(p=0.4, inplace=False)
  (layers): Sequential(
    (0): Linear(in_features=12, out_features=200, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=200, out_features=100, bias=True)
    (5): ReLU(inplace=True)
    (6): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.4, inplace=False)
    (8): Linear(in_features=100, out_features=50, bias=True)
    (9): ReLU(inplace=True)
    (10): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Dropout(p=0.4, inplace=False)
    (12): Linear(in_features=50, out_features=4, bias=True)
  )
)


In [37]:
# 4.nn.Sequential을 이용해 nn.Linear, nn.ReLU, nn.Dropout 세 층이 있는 모델을 만들어 보자.
class Model2(nn.Module):
  def __init__(self, input_size, output_size):
    super().__init__()
    self.layer = nn.Sequential(nn.Linear(input_size, output_size), nn.ReLU(), nn.Dropout())

  def forward(self, x):
    return self.layer(x)

model2 = Model2(5, 10)
x = torch.ones(5)
model2(x)

tensor([0.0000, 0.0000, 0.0000, 0.8492, 0.0000, 0.0000, 1.0357, 1.4312, 1.2888,
        0.6242], grad_fn=<MulBackward0>)

In [38]:
class Model2(nn.Module):
  def __init__(self, input_size, output_size):
    super().__init__()
    empty_list = []
    for i in range(5):
      empty_list.append(nn.Linear(input_size, input_size))
      empty_list.append(nn.ReLU())
      empty_list.append(nn.Dropout())

    self.layer = nn.Sequential(*empty_list)

  def forward(self, x):
    return self.layer(x)

model2 = Model2(5, 10)
x = torch.ones(5)
model2(x)

tensor([0.0000, 0.0000, 0.5562, 0.6456, 0.0000], grad_fn=<MulBackward0>)

In [39]:
# 5.nn.Module을 이용해 자신만의 모델을 만들고 실행해보자. 단 해당 모델은 nn.Linear, nn.ReLU 를 5개 이상 포함해야 한다.
class Model3(nn.Module):
  def __init__(self, input_size, output_size):
    super().__init__()
    empty_list = []
    for i in range(5):
      empty_list.append(nn.Linear(10, 10))
      empty_list.append(nn.ReLU())

    empty_list = [nn.Linear(input_size, 10)] + empty_list + [nn.Linear(10, output_size)]
    self.layer = nn.Sequential(*empty_list)

  def forward(self, x):
    return self.layer(x)

model3 = Model3(5, 20)
x = torch.ones(5)
# model3(x)
model3

Model3(
  (layer): Sequential(
    (0): Linear(in_features=5, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): ReLU()
    (3): Linear(in_features=10, out_features=10, bias=True)
    (4): ReLU()
    (5): Linear(in_features=10, out_features=10, bias=True)
    (6): ReLU()
    (7): Linear(in_features=10, out_features=10, bias=True)
    (8): ReLU()
    (9): Linear(in_features=10, out_features=10, bias=True)
    (10): ReLU()
    (11): Linear(in_features=10, out_features=20, bias=True)
  )
)

#### 2-5. 모델 훈련을 위한 준비

In [40]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [41]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

#### 2-6. 모델 훈련

In [3]:
epochs = 500
aggregated_losses = []
train_outputs = train_outputs.to(device=device, dtype=torch.float32)
for i in range(epochs):
    i += 1
    y_pred = model(categorical_train_data)
    single_loss = loss_function(y_pred, train_outputs)
    aggregated_losses.append(single_loss)

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

    optimizer.zero_grad()
    single_loss.backward()
    optimizer.step()

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

NameError: name 'train_outputs' is not defined

#### 2-7. 결과 확인

In [45]:
test_outputs = test_outputs.to(device=device, dtype=torch.float32)
with torch.no_grad():
    y_val = model(categorical_test_data)
    loss = loss_function(y_val, test_outputs)
print(f'Loss: {loss:.8f}')

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [46]:
print(y_val[:5])

tensor([[ 3.0474e-01, -2.2798e+00, -2.6482e-01,  5.6713e-01],
        [ 1.3500e-03, -3.4071e-01,  4.5678e-01, -1.4195e-01],
        [-3.3419e-01,  9.1897e-01,  3.2722e-01,  2.3233e-01],
        [ 2.8110e-01,  2.7785e-01,  3.3806e-01, -7.3798e-01],
        [-6.3807e-01, -1.5414e-01,  1.7499e-01, -1.4320e-01]])


In [47]:
y_val = torch.argmax(y_val, dim=1)
print(y_val[:5])

tensor([3, 2, 1, 2, 2])


In [48]:
# 테스트 데이터의 5번째부터 9번째까지의 데이터에 대해 모델의 예측을 수행해보자.
with torch.no_grad():
    y_val = model(categorical_test_data)
print(y_val[5:10])
print(torch.argmax(y_val, 1)[5:10])

tensor([[ 0.2723, -0.6366, -0.3024, -0.8999],
        [-0.7885, -0.2322, -0.9505,  0.5773],
        [ 0.7752,  0.0667,  0.3994,  0.4739],
        [ 0.6053,  0.1702,  0.4125,  1.0118],
        [-0.3931,  0.0025, -0.1681,  1.0394]])
tensor([0, 3, 0, 3, 3])


In [49]:
# 테스트 데이터 중 모델의 예측상 vgood (인덱스 기준 3) 인 차량은 얼마나 되는가?
count = 0
for index in torch.argmax(y_val, 1):
  if index == 3:
    count = count + 1
print(count)

97


#### 2-8. 정확도, 정밀도, 재현율

In [50]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(classification_report(torch.argmax(test_outputs, dim=1), torch.argmax(y_val, dim=1)))
print(accuracy_score(torch.argmax(test_outputs, dim=1), torch.argmax(y_val, dim=1)))

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [54]:
# 우리가 만들고 훈련시킨 모델은 다음과 같은 파라미터로 정의되었다. (embedding_size, output_size, layers, p)
# (categorical_embedding_sizes, 4, [200,100,50], p=0.4)
# 모델을 더 깊게 하려면 어떤 파라미터를 바꿔야 할까?
# dropout을 더 강화하려면 어떤 파라미터를 바꿔야 할까?
# 각 항목에 해당하는 모델을 선언하고 훈련시킨 후 결과를 확인해 보자.

In [61]:
model = Model(categorical_embedding_sizes, 4, [200,100,50,50,50], p=0.4)
print(model)

Model(
  (all_embeddings): ModuleList(
    (0): Embedding(4, 2)
    (1): Embedding(4, 2)
    (2): Embedding(4, 2)
    (3): Embedding(3, 2)
    (4): Embedding(3, 2)
    (5): Embedding(3, 2)
  )
  (embedding_dropout): Dropout(p=0.4, inplace=False)
  (layers): Sequential(
    (0): Linear(in_features=12, out_features=200, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=200, out_features=100, bias=True)
    (5): ReLU(inplace=True)
    (6): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.4, inplace=False)
    (8): Linear(in_features=100, out_features=50, bias=True)
    (9): ReLU(inplace=True)
    (10): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Dropout(p=0.4, inplace=False)
    (12): Linear(in_features=50, out_features=50, bias=True)
    (13): ReL

In [55]:
# 최대한 기억에 의존해서 모델을 학습시키는 코드를 다시 짜보자.
# 힌트: 학습은 아래와 같은 기능을 포함해야 한다.
# loss.backward(), loss_function(y_pred, train_outputs), optimizer.zero_grad(), optimizer.step()

In [64]:
train_outputs = train_outputs.to(device=device, dtype=torch.float32)
for i in range(100):
    y_pred = model(categorical_train_data)
    single_loss = loss_function(y_pred, train_outputs)
    optimizer.zero_grad()
    single_loss.backward()
    optimizer.step()

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!